# Welcome! 
This IPython notebook does a sensitivity analysis of the mobility model for the water access research project.

Import Packages

In [1]:
# Importing necessary modules
import os  # Provides functions for interacting with the operating system
import sys  # Provides access to some variables used by the interpreter
from pathlib import Path  # Used for filesystem path operations

# Resolve the current path of the notebook. Path().resolve() returns the absolute path
# of the current working directory (where this notebook is located).
notebook_path = Path().resolve()

# Determine the project root directory. Since 'notebook_path' points to the 'scripts'
# directory, its parent will be the root directory of your project (which contains both 'scripts' and 'src').
project_root = notebook_path.parent

# Append the project root directory to sys.path. This step is crucial as it allows
# Python to find and import modules from the 'src' directory, which is located at the project root.
# It essentially adds the project root to the list of directories where Python looks for modules.
sys.path.append(str(project_root))

# Importing standard and third-party libraries necessary for your script.
# These libraries are likely installed in your Conda environment or Python's site-packages.
import pandas as pd  # Data manipulation and analysis library
import numpy as np  # Library for numerical operations
import plotly  # Graphing library
import plotly.express as px  # Simplified interface for Plotly
from plotly.subplots import make_subplots  # Function to create subplots with Plotly
import plotly.graph_objects as go  # For creating different types of figures in Plotly

# Importing custom modules from the 'src' directory.
# Thanks to modifying sys.path, Python can now find these modules.
import src.mobility_module as mm  # Custom module for mobility-related functionalities
import src.plotting_tools_water_access as pt  # Custom module for plotting tools specific to water access
import seaborn as sns 
sns.set_theme()
import matplotlib.pyplot as plt
plt.style.use("https://raw.githubusercontent.com/allfed/ALLFED-matplotlib-style-sheet/main/ALLFED.mplstyle")



Import data from csv files

In [2]:
######################
#### Import Data #####
file_path_params = "../data/lookup tables/mobility-model-parameters.csv"
file_path_sens_vars = "../data/lookup tables/Sensitivity Analysis Variables.csv"


allHPV_param_df = pd.read_csv(file_path_params)
sens_df = pd.read_csv(file_path_sens_vars)


Set model for sensitivity analysis

In [3]:
# Lankford for walking, Martin for cycling
MODEL = "Lankford"

if MODEL == "Martin":
    model_number = 2
    filter_value = "Bicycle"
elif MODEL == "Lankford":
    model_number = 3
    filter_value = "Buckets"
    sens_df.loc[sens_df["Short Name"] == "HPV Weight", "Expected Min"] = 0.5
    sens_df.loc[sens_df["Short Name"] == "HPV Weight", "Expected Max"] = 3
    sens_df.loc[sens_df["Short Name"] == "HPV Weight", "Default Value"] = 1
print(sens_df.to_markdown())

|    | Short Name                        | Long Name                          | Units   | Affected Models          | Notes                                                                                        |   Default Value |   Expected Max |   Expected Min |   Plotting Max |   Plotting Min | Status   |   Unnamed: 11 | Guess of importance   |
|---:|:----------------------------------|:-----------------------------------|:--------|:-------------------------|:---------------------------------------------------------------------------------------------|----------------:|---------------:|---------------:|---------------:|---------------:|:---------|--------------:|:----------------------|
|  0 | Coefficient of Rolling Resistance | coefficient of rolling resistance  |         | sprott, cycling          | Related to road quality                                                                      |           0.003 |          0.03  |       0.002    |         0.0435 |       0.0015   | test

Initiliase Velocity Kgs, define colours for graphing

In [4]:
graph_colours = ["#3D87CB", "#F0B323", "#DC582A", "#674230", "#3A913F", "#75787B"]
full_result_dict = {}
plot_dict = {}
df_large = pd.DataFrame()


### FIlter Data
col = "Name"
param_df = allHPV_param_df.loc[(allHPV_param_df[col] == filter_value)]

for i in range(0, len(sens_df)):

    # # initialise variables, options, hpv and Velocity Kgs classes, populated with data via 'init' functions
    mo = mm.model_options()
    mv = mm.model_variables()
    met = mm.MET_values(mv, country_weight=60, met = 3.5, use_country_specific_weights=False)
    hpv = mm.HPV_variables(param_df, mv)
    mr = mm.model_results(hpv, mo)

    # select model (bike...)
    mo.model_selection = model_number


    ###### Start big if statement area
    ##### SENSITIVITY
    res = 7

    # set up variables for sensitivity analysis
    plot_min = sens_df.iloc[i]["Plotting Min"]
    plot_max = sens_df.iloc[i]["Plotting Max"]
    minval = sens_df.iloc[i]["Expected Min"]
    maxval = sens_df.iloc[i]["Expected Max"]
    def_val = sens_df.iloc[i]["Default Value"]
    var_string = sens_df.iloc[i]["Short Name"] # change
    var_units = sens_df.iloc[i]["Units"] # change

    # create linear space of weights
    phase_space = np.linspace(
        start=plot_min,  # define linear space
        stop=plot_max,  # define maximum value for linear space
        num=res,  # how many data points?
        endpoint=True,
        retstep=False,
        dtype=None,
    )
    # include the default, max, and min values seperatly so they are easy to see graphically
    # phase_space = np.append(phase_space,hpv.load_limit.flatten()[0])
    phase_space = np.append(phase_space,def_val)
    phase_space = np.append(phase_space,minval)
    phase_space = np.append(phase_space,maxval)

    trip_velocity_mean = []  # create empty list to place variables in to in loop
    unloaded_velocity_mean = []  # create empty list to place variables in to in loop
    loaded_velocity_mean = []  # create empty list to place variables in to in loop
    velocitykgs = []  # create empty list to place variables in to in loop
    water_ration_kms = []  # create empty list to place variables in to in loop

    for var_test in phase_space:
        ## THIS WILL NEED A FAT IF STATEMENT AS WE NEED TO CHANGE TARGET of the assignment AND the value AND there are differening data storgae types (i.e numpy 3 dims, scalar etc)
        if var_string == "Coefficient of Rolling Resistance":
            hpv.Crr = np.array([[[var_test]]]) # change
        elif var_string == "Load Limit":
            hpv.load_limit = np.array([[[var_test]]]) 
        elif (var_string == "Practical Limit Cycling" and mo.model_selection == 2):
            hpv.practical_limit = np.array([[[var_test]]])
        elif (var_string == "Practical Limit Walking" and mo.model_selection == 3):
            hpv.practical_limit = np.array([[[var_test]]])
        elif var_string == "Reference Area":
            mv.A = var_test 
        elif var_string == "Drag Coefficient":
            mv.C_d = var_test 
        elif var_string == "Efficiency":
            mv.eta = var_test 
        elif var_string == "Air Density":
            mv.ro = var_test 
        elif var_string == "MET budget":
            met = mm.MET_values(mv, country_weight=60, met = var_test, use_country_specific_weights=False)
        elif var_string == "T_hours":
            mv.t_hours = var_test 
        elif var_string == "HPV Weight":
            hpv.m_HPV_only = np.array([[[var_test]]]) # change
        elif var_string == "Water Ration":
            mv.waterration = var_test 
        elif var_string == "Human Weight":
            mv.m1 = var_test 
        elif var_string == "Human Power Output":
            mv.P_t = var_test 
        
        
        # # hpv.load_limit = np.array([[[var_test]]]) # change
        # mv.C_d = var_test # change

        ######## Numerical MODEL ########
        mr.v_load_matrix3d, mr.load_matrix3d = mm.mobility_models.numerical_mobility_model(
            mr, mv, mo, met, hpv
        )
        ####### Organise Velocity Kgs #######
        mr.load_results(hpv, mv, mo)

        maximum_vel_kg_per_slope = np.amax(mr.velocitykgs, axis=2)
        mean_vel_kg_per_slope = np.mean(maximum_vel_kg_per_slope)

        # append to list
        trip_velocity_mean.append(np.mean(mr.v_avg_matrix3d))
        unloaded_velocity_mean.append(np.mean(mr.v_unload_matrix3d))
        loaded_velocity_mean.append(np.mean(mr.v_load_matrix3d))
        velocitykgs.append(mean_vel_kg_per_slope)
        water_ration_kms.append(mean_vel_kg_per_slope/mv.waterration*mr.t_secs/1000) # [kg][m/s] * [s] * 1/[kg] * 1/1000 = [km]. Divide by distance to water, and accomodate for household size to see if needs are met.



    small_marker_size = 10
    med_marker_size = 20
    large_marker_size = 50
    data = {'Velocity Kgs': velocitykgs,'Water Ration Kms': water_ration_kms, 'Mean Trip Velocity': trip_velocity_mean, 'Mean Unloaded Velocity': unloaded_velocity_mean, 'Mean Loaded Velocity': loaded_velocity_mean}
    df_results = pd.DataFrame(data)

    df_results["Adjusted Result"] =  df_results["Velocity Kgs"] - df_results.at[res,'Velocity Kgs']
    df_results["Adjusted Water Ration Kms"] =  df_results["Water Ration Kms"] - df_results.at[res,'Water Ration Kms']
    df_results["Variable"] = phase_space
    df_results["DataType"] = "Sensitivity"
    df_results["MarkerSize"] = small_marker_size
    df_results["Name"] = var_string
    df_results.at[res,'DataType'] = "Default"
    df_results.at[res+1,'DataType'] = "Minimum Expected"
    df_results.at[res+2,'DataType'] = "Maximum Expected"
    df_results.at[res+1,'MarkerSize'] = med_marker_size
    df_results.at[res+2,'MarkerSize'] = med_marker_size
    df_results.at[res,'MarkerSize'] = large_marker_size


    full_result_dict[var_string] = df_results
    df_large = pd.concat([df_large, df_results], sort=False)



In [5]:

# for i in range(0, len(sens_df)):
primary_graph_value = "Water Ration Kms"
max_plot_y = df_large[primary_graph_value].max()*1.10
min_plot_y = df_large[primary_graph_value].min()*0.

graph_colours = ["#3D87CB", "#F0B323", "#DC582A", "#674230", "#3A913F", "#75787B"]


for i in range(0, len(sens_df)):

    fig1 = px.scatter(
        full_result_dict[sens_df.iloc[i]["Short Name"]],
        x="Variable",
        y=primary_graph_value,
        color='DataType',
        color_discrete_sequence=graph_colours,
        size='MarkerSize',
        title=f'Effect of {sens_df.iloc[i]["Short Name"]} on model Velocity Kgs',
        hover_data=[
            "Velocity Kgs",
            'Water Ration Kms',
            'Mean Trip Velocity',
            'Mean Loaded Velocity',
            'Mean Unloaded Velocity',
        ]


    ).update_layout(
        yaxis_title=primary_graph_value,
        xaxis_title=(f'{sens_df.iloc[i]["Short Name"]}  <i>{sens_df.iloc[i]["Units"]}</i>'),
    )
    fig1.update_yaxes(range = [min_plot_y, max_plot_y])

    fig1 = pt.format_plotly_graphs(fig1)

    fig1.show()
    plot_dict[var_string]= fig1
    # filename_string = var_string + ".html"
    # plotly.offline.plot(fig1, filename=filename_string)

In [6]:
df_large[primary_graph_value].max()*1.10
df_large[primary_graph_value].min()*0.90

0.4338477432507915

In [7]:
summary_df = df_large[(df_large.DataType == "Minimum Expected") | (df_large.DataType == "Maximum Expected") | (df_large.DataType == "Default")]
# summary_df = df_large[(df_large.DataType == "Minimum Expected") | (df_large.DataType == "Maximum Expected")]

# Rename 'T_hours'' and "Crr"
summary_df['Name'] = summary_df['Name'].replace({
   'T_hours': 'Time gathering water [hours]',
   'Reference Area': 'Reference area [m^2]',
   'Coefficient of Rolling Resistance': 'Coefficient of rolling resistance',
   'Water Ration': 'Water ration [L/day]',
   'Load Limit': 'Rated capacity [kg]',
   'Practical Limit Cycling': 'Practical limit (cycling) [kg]',
   'Practical Limit Walking': 'Practical limit (walking) [kg]',
   'Human Weight': 'Human mass [kg]',
   'Human Power Output': 'Power output [W]',
   'MET budget': 'Energy expenditure [MET]',
   'HPV Weight': 'Vehicle/container mass [kg]',
   'Air Density': 'Air density [kg/m^3]',
   'Drag Coefficient': 'Drag coefficient',
   })


# Pivot DataFrame so that each 'DataType' value becomes a column
pivot_df = summary_df.pivot(index='Name', columns='DataType', values='Adjusted Water Ration Kms')

# Create a new column 'abs_diff' that contains the absolute differences
pivot_df['abs_diff'] = np.abs(pivot_df['Maximum Expected'] - pivot_df['Minimum Expected'])

# Reset index
pivot_df.reset_index(inplace=True)

# Sort DataFrame by 'abs_diff' in descending order
pivot_df = pivot_df.sort_values('abs_diff', ascending=False)

# Merge pivot_df with summary_df to get the original structure but sorted
summary_df = pd.merge(pivot_df['Name'], summary_df, on='Name')

# Define color mapping
color_discrete_map = {'Minimum Expected': '#3D87CB', 'Maximum Expected': '#004680', 'Default': None} 

# Filter based on model

if mo.model_selection == 2:
   summary_df = summary_df[
         ~summary_df.Name.isin(
            [
               "Energy expenditure [MET]",
               'Practical limit (walking) [kg]',
            ]
         )
      ]
elif mo.model_selection == 3:
   # filter out reference area, load limit, human power output, efficiency, drag coefficient, coefficient of rolling resistance, and air density
   if mo.model_selection == 3:
      summary_df = summary_df[
         ~summary_df.Name.isin(
            [
               # "Load Limit",
               'Reference area [m^2]', 
               'Practical limit (cycling) [kg]', 
               'Power output [W]', 
               'Efficiency', 
               'Drag coefficient', 
               'Coefficient of rolling resistance', 
               'Air density [kg/m^3]',
               'Rated capacity [kg]',
            ]
         )
      ]

/tmp/ipykernel_439626/1583398998.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
# Create plot

# Filter out 'Default' rows from summary_df
filtered_summary_df = summary_df[summary_df['DataType'] != 'Default']

# Create plot
fig = px.bar(filtered_summary_df, y="Name", x="Adjusted Water Ration Kms", color="DataType", color_discrete_map=color_discrete_map,
             labels={
                "Name": "",
                "Adjusted Water Ration Kms": "Adjusted Water Ration Kms [15 L.km]",
                "DataType": "Input Value"
             },)

# Extract Default, Min, and Max values
default_values = summary_df[summary_df['DataType'] == 'Default']
min_expected_values = summary_df[summary_df['DataType'] == 'Minimum Expected']
max_expected_values = summary_df[summary_df['DataType'] == 'Maximum Expected']

# Function to determine xshift based on bar position
def get_xshift(row, default_values):
    default_value = default_values[default_values['Name'] == row['Name']]['Adjusted Water Ration Kms'].values[0]
    if row['Adjusted Water Ration Kms'] < default_value:
        return -25  # Shift to the right
    else:
        return 25  # Shift to the left

# Add annotations for Default values
# for i, row in default_values.iterrows():
#     fig.add_annotation(
#         x=row['Adjusted Water Ration Kms'],
#         y=row['Name'],
#         text=f"{row['Variable']}",
#         showarrow=False,
#         font=dict(color="black"),
#         bgcolor="white",
#     )

# Add annotations for Min values
for i, row in min_expected_values.iterrows():
    fig.add_annotation(
        x=row['Adjusted Water Ration Kms'],
        y=row['Name'],
        text=f"{row['Variable']: .3g}",
        showarrow=False,
        font=dict(color="#3D87CB"),
        # bgcolor="white",
        xshift=get_xshift(row, default_values),
    )

# # Add annotations for Max values
for i, row in max_expected_values.iterrows():
    fig.add_annotation(
        x=row['Adjusted Water Ration Kms'],
        y=row['Name'],
        text=f"{row['Variable']: .3g}",
        showarrow=False,
        font=dict(color="#004680"),
        # bgcolor="white",
        xshift=get_xshift(row, default_values),
    )


fig.update_layout(
            width=1000,
            height=500,
            font=dict(size=16),
            margin=dict(l=275, r=50, t=50, b=50),  # Adjust margins to ensure the plotting area remains constant
        )

fig = pt.format_plotly_graphs(fig)

fig.show()


if MODEL == "Martin":
    fig.write_image("../results/sensitivity_analysis_a.png", width=1000, height=500)
elif MODEL == "Lankford":
    fig.write_image("../results/sensitivity_analysis_b.png", width=1000, height=500)

In [9]:
# summary_df["Name"].to_csv("listvars.csv")
summary_df

,Name,Velocity Kgs,Water Ration Kms,Mean Trip Velocity,Mean Unloaded Velocity,Mean Loaded Velocity,Adjusted Result,Adjusted Water Ration Kms,Variable,DataType,MarkerSize
0,Energy expenditure [MET],25.596990,33.788027,1.413133,1.573901,1.252365,0.000000,0.000000,4.5,Default,50
1,Energy expenditure [MET],11.478227,15.151259,0.662767,0.769432,0.556102,-14.118763,-18.636767,3.0,Minimum Expected,20
2,Energy expenditure [MET],39.715753,52.424794,2.163499,2.378371,1.948627,14.118763,18.636767,6.0,Maximum Expected,20
3,Water ration [L/day],16.184481,21.363515,0.912889,1.037588,0.788190,0.000000,0.000000,15.0,Default,50
4,Water ration [L/day],16.184481,32.045272,0.912889,1.037588,0.788190,0.000000,10.681757,10.0,Minimum Expected,20
5,Water ration [L/day],16.184481,10.681757,0.912889,1.037588,0.788190,0.000000,-10.681757,30.0,Maximum Expected,20
6,Human mass [kg],16.184481,21.363515,0.912889,1.037588,0.788190,0.000000,0.000000,62.0,Default,50
7,Human mass [kg],19.749208,26.068955,1.112989,1.266749,0.959229,3.564727,4.705440,55.2,Minimum Expected,20
8,Human mass [kg],4.736235,6.251830,0.282866,0.334852,0.230881,-11.448246,-15.111685,99.4,Maximum Expected,20
9,Time gathering water [hours],16.184481,21.363515,0.912889,1.037588,0.788190,0.000000,0.000000,5.5,Default,50
